In [85]:
%reload_ext autoreload
%load_ext autotime
%reload_ext line_profiler
%autoreload 2

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import sys
sys.path.insert(0,'../../')

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import glob
import re

from scripts.manipulate import normalize_cols, padronize_str

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 46 ms (started: 2021-04-19 14:25:07 -03:00)


In [ ]:
new_df = pd.DataFrame()

In [25]:
new_df = pd.DataFrame(df.columns.to_list(), columns=[ano])

time: 28.2 ms (started: 2021-04-19 11:23:07 -03:00)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2010,orgao,uo,unidade_gestora,fonte_de_recursos,funcao,sub_funcao,programa,acao,funcional_programatica,elemento,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,pago_restos,unnamed:_16


time: 42.5 ms (started: 2021-04-19 11:23:38 -03:00)


In [94]:
## CHECA SE ARQUIVOS SAO CONSISTENTES ENTRE OS ANOS
files = glob.glob('../../data/orcamento_consolidado/execucao/*.csv')
df_cols = pd.DataFrame()
df_orcamento = pd.DataFrame()
for file in files:
    ano = re.findall(r'\d\d\d\d', file)[0]
    print(ano)
    print(file)

  
    df = pd.read_csv(file, encoding='latin1', dtype=str,  index_col=False)
    df.columns = normalize_cols(df.columns)
    df = df[df['orgao'].notnull()].copy()
    df['ano'] = int(ano)
    
    numeric_columns = [
    "dotacao_inicial",
    "dotacao_atual",
    "empenhado",
    "liquidado",
    "pago",
    "pago_restos",
    ]
    
    for col in numeric_columns:
        df[col] = df[col].str.replace('.','').str.replace(',','.')
        df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
    
    str_cols = [
        "orgao",
        "uo",
        "unidade_gestora",
        "fonte_de_recursos",
        "funcao",
        "sub_funcao",
        "programa",
        "acao",
        "funcional_programatica",
        "elemento",
    ]
    for col in str_cols:
        df[col] = padronize_str(df[col])
    
    df_orcamento = pd.concat([df_orcamento,df])
    
    
    
    all_cols[ano]={f"{col}":"" for col in df.columns.to_list()}
    
    new_df = pd.DataFrame(df.columns.to_list(), columns=[ano])
    df_cols = pd.concat([df_cols, new_df],1)
    
# df_orcamento = df_orcamento.drop(['unnamed:_16','date'],1)

2010
../../data/orcamento_consolidado/execucao/orcamento_2010.csv
2011
../../data/orcamento_consolidado/execucao/orcamento_2011.csv
2012
../../data/orcamento_consolidado/execucao/orcamento_2012.csv
2013
../../data/orcamento_consolidado/execucao/orcamento_2013.csv
2014
../../data/orcamento_consolidado/execucao/orcamento_2014.csv
2015
../../data/orcamento_consolidado/execucao/orcamento_2015.csv
2016
../../data/orcamento_consolidado/execucao/orcamento_2016.csv
2017
../../data/orcamento_consolidado/execucao/orcamento_2017.csv
2018
../../data/orcamento_consolidado/execucao/orcamento_2018.csv
2019
../../data/orcamento_consolidado/execucao/orcamento_2019.csv
2020
../../data/orcamento_consolidado/execucao/orcamento_2020.csv
2021
../../data/orcamento_consolidado/execucao/orcamento_2021.csv
time: 9.09 s (started: 2021-04-19 14:33:37 -03:00)


time: 157 ms (started: 2021-04-19 14:39:47 -03:00)


In [120]:
df_orcamento.groupby(by=['ano'],as_index=False).sum()[['ano','dotacao_inicial']].sort_values(by='ano', ascending=False)

,ano,dotacao_inicial
11,2021,2.772163e+11
10,2020,2.697748e+11
9,2019,2.608308e+11
8,2018,2.459258e+11
7,2017,2.342283e+11
6,2016,2.341096e+11
5,2015,2.310481e+11
4,2014,2.121914e+11
3,2013,1.928443e+11
2,2012,1.740679e+11


time: 120 ms (started: 2021-04-19 14:45:28 -03:00)


In [130]:
df_orcamento['funcao'].unique()

array(['01 - LEGISLATIVA', '02 - JUDICIARIA', '12 - EDUCACAO',
       '10 - SAUDE', '19 - CIENCIA E TECNOLOGIA',
       '23 - COMERCIO E SERVICOS', '13 - CULTURA',
       '09 - PREVIDENCIA SOCIAL', '20 - AGRICULTURA', '26 - TRANSPORTE',
       '14 - DIREITOS DA CIDADANIA', '21 - ORGANIZACAO AGRARIA',
       '08 - ASSISTENCIA SOCIAL', '06 - SEGURANCA PUBLICA',
       '04 - ADMINISTRACAO', '28 - ENCARGOS ESPECIAIS', '11 - TRABALHO',
       '16 - HABITACAO', '18 - GESTAO AMBIENTAL',
       '03 - ESSENCIAL A JUSTICA', '17 - SANEAMENTO', '25 - ENERGIA',
       '27 - DESPORTO E LAZER', '24 - COMUNICACOES',
       '99 - RESERVA DE CONTINGENCIA', '22 - INDUSTRIA', '15 - URBANISMO'],
      dtype=object)

time: 58 ms (started: 2021-04-19 14:55:39 -03:00)


In [131]:
### elementos da previdencia
funcoes_interesse = [
    '10 - SAUDE',
    '12 - EDUCACAO',
    '19 - CIENCIA E TECNOLOGIA',
    '09 - PREVIDENCIA SOCIAL',
    '06 - SEGURANCA PUBLICA',
]
mask = df_orcamento['funcao'].isin(funcoes_interesse)
df_orcamento[mask].groupby(by=['ano','funcao'],as_index=False).sum()[['ano','funcao','dotacao_inicial']].sort_values(by=['funcao','ano'], ascending=False)

,ano,funcao,dotacao_inicial
59,2021,19 - CIENCIA E TECNOLOGIA,1.513406e+09
54,2020,19 - CIENCIA E TECNOLOGIA,1.965812e+09
49,2019,19 - CIENCIA E TECNOLOGIA,1.877747e+09
44,2018,19 - CIENCIA E TECNOLOGIA,1.706173e+09
39,2017,19 - CIENCIA E TECNOLOGIA,1.641950e+09
34,2016,19 - CIENCIA E TECNOLOGIA,1.587571e+09
29,2015,19 - CIENCIA E TECNOLOGIA,1.516750e+09
24,2014,19 - CIENCIA E TECNOLOGIA,1.436620e+09
19,2013,19 - CIENCIA E TECNOLOGIA,1.447495e+09
14,2012,19 - CIENCIA E TECNOLOGIA,1.388150e+09


time: 150 ms (started: 2021-04-19 14:55:50 -03:00)


In [141]:
grupo = {
    '31' : 'PESSOAL E ENCARGOS SOCIAIS',
    '32' : 'JUROS E ENCARGOS DA DIVIDA',
    '33' : 'OUTRAS DESPESAS CORRENTES',
    '44' : 'INVESTIMENTOS',
    '45' : 'INVERSOES FINANCEIRAS',
    '46' : 'AMORTIZACAO DE DIVIDA',
    '99' : 'RESERVA DE CONTINGENCIA'
}

df_orcamento['grupo'] = df_orcamento['elemento'].apply(lambda x: x[:2]).replace(grupo)



df_orcamento.groupby(by=['ano','grupo'],as_index=False).sum()[['ano','grupo','dotacao_inicial']].sort_values(by=['grupo','ano'], ascending=False)

,ano,grupo,dotacao_inicial
84,2021,RESERVA DE CONTINGENCIA,5.100000e+07
77,2020,RESERVA DE CONTINGENCIA,4.999996e+07
70,2019,RESERVA DE CONTINGENCIA,5.000000e+07
63,2018,RESERVA DE CONTINGENCIA,4.650000e+07
56,2017,RESERVA DE CONTINGENCIA,2.000000e+07
49,2016,RESERVA DE CONTINGENCIA,1.000000e+07
42,2015,RESERVA DE CONTINGENCIA,1.000000e+07
35,2014,RESERVA DE CONTINGENCIA,1.000000e+07
28,2013,RESERVA DE CONTINGENCIA,1.000000e+07
21,2012,RESERVA DE CONTINGENCIA,1.000000e+07


time: 387 ms (started: 2021-04-19 15:03:01 -03:00)
